In [52]:
import random, re, math
import numpy as np
import pandas as pd
import sympy as sp
import tensorflow as tf
import mpmath as mp

from tensorflow.keras.layers import (
    Input, TextVectorization, Embedding, LSTM,
    Dense, Concatenate, Lambda
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TerminateOnNaN
from sklearn.model_selection import train_test_split
from functools import lru_cache

# Symbol for Sympy
x = sp.symbols('x')

# Hyperparameters
SEQ_LEN    = 64
BATCH      = 64
LR         = 1e-4
EPOCHS     = 20
INT_TOL    = 1e-6


In [53]:
df_real = pd.read_csv(
    "functions2.csv", header=None,
    names=["function","lower","upper","true_raw"]
)
df_real["lower"]     = pd.to_numeric(df_real["lower"], errors="coerce")
df_real["upper"]     = pd.to_numeric(df_real["upper"], errors="coerce")
df_real["true_raw"]  = pd.to_numeric(df_real["true_raw"], errors="coerce")
df_real.dropna(subset=["lower","upper","true_raw"], inplace=True)
df_real.reset_index(drop=True, inplace=True)


In [54]:
@lru_cache(maxsize=None)
def make_random_poly(deg):
    coeffs = [random.uniform(-5,5) for _ in range(deg+1)]
    return sum(c*x**i for i,c in enumerate(coeffs))

@lru_cache(maxsize=None)
def integrate_sympy(expr_str, a, b):
    expr = sp.sympify(expr_str.replace('^','**'))
    return float(sp.integrate(expr, (x, a, b)))

rows = []
for _ in range(5000):
    d = random.randint(1,5)
    e = make_random_poly(d)
    a, b = random.uniform(-3,0), random.uniform(0,3)
    s = integrate_sympy(str(e).replace('**','^'), a, b)
    rows.append({
        "function":  str(e).replace('**','^'),
        "lower":     a,
        "upper":     b,
        "true_raw":  s
    })
df_synth = pd.DataFrame(rows)


In [55]:
df = pd.concat([df_real, df_synth], axis=0).sample(frac=1.0, random_state=42).reset_index(drop=True)
print("Total samples:", len(df))


Total samples: 207252


In [56]:
y_mean, y_std = df["true_raw"].mean(), df["true_raw"].std()
df["y_norm"]  = (df["true_raw"] - y_mean) / y_std

l_mean, l_std = df["lower"].mean(), df["lower"].std()
u_mean, u_std = df["upper"].mean(), df["upper"].std()
df["lower_n"] = (df["lower"] - l_mean) / l_std
df["upper_n"] = (df["upper"] - u_mean) / u_std
@lru_cache(maxsize=None)
def poly_feats_cached(f_str):
    try:
        expr = sp.sympify(f_str.replace('^','**'))
        poly = sp.Poly(expr, x)
        return tuple(float(poly.coeff_monomial(x**i)) for i in range(6))
    except:
        return (0.0,)*6

df["poly_feats"] = df["function"].apply(poly_feats_cached)


/Users/sbhat/Library/Python/3.9/lib/python/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [57]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

vectorizer = TextVectorization(
    output_mode="int",
    output_sequence_length=SEQ_LEN,
    standardize=lambda s: tf.strings.regex_replace(tf.strings.lower(s), r"\^","**"),
    split="character"
)
vectorizer.adapt(train_df["function"].values)


In [58]:
def make_ds(ddf, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((
        {
            "func_input":  ddf["function"].values,
            "lower_input": ddf["lower_n"].values.astype(np.float32),
            "upper_input": ddf["upper_n"].values.astype(np.float32),
            "poly_feats":  np.stack(ddf["poly_feats"].values).astype(np.float32),
        },
        ddf["y_norm"].values.astype(np.float32),
    ))
    if shuffle: ds = ds.shuffle(buffer_size=len(ddf))
    return ds.batch(BATCH).cache().prefetch(tf.data.AUTOTUNE)

train_ds = make_ds(train_df, shuffle=True)
val_ds   = make_ds(val_df,   shuffle=False)


/var/folders/sp/h4fz4p0j7_nbfd62d4_qw3b80000gq/T/ipykernel_31455/3183424454.py:7: RuntimeWarning: overflow encountered in cast
  "poly_feats":  np.stack(ddf["poly_feats"].values).astype(np.float32),


In [59]:
f_in   = Input(shape=(), dtype=tf.string, name="func_input")
toks   = vectorizer(f_in)
toks_i = Lambda(lambda t: tf.cast(t, tf.int32), name="cast_int")(toks)
x      = Embedding(input_dim=vectorizer.vocabulary_size(), output_dim=32, mask_zero=True)(toks_i)
x      = LSTM(32)(x)

# — bounds branch —
l_in   = Input(shape=(1,), dtype=tf.float32, name="lower_input")
u_in   = Input(shape=(1,), dtype=tf.float32, name="upper_input")
b      = Concatenate()([l_in, u_in])
b      = Dense(16, activation="relu")(b)
b      = Dense(8,  activation="relu")(b)

# — poly‐coeff branch —
p_in   = Input(shape=(6,), dtype=tf.float32, name="poly_feats")
p      = Dense(32, activation="relu")(p_in)
p      = Dense(16, activation="relu")(p)

# — merge & output —
m      = Concatenate()([x, b, p])
m      = Dense(64, activation="relu")(m)
m      = Dense(32, activation="relu")(m)
out    = Dense(1, activation="linear", name="pred")(m)

model = Model(inputs=[f_in, l_in, u_in, p_in], outputs=out)
model.compile(
    optimizer=Adam(LR, clipnorm=1.0),
    loss=tf.keras.losses.Huber(),
    metrics=["mae"]
)
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ func_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_vectorization… │ (None, 64)        │          0 │ func_input[0][0]  │
│ (TextVectorization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lower_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ upper_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_int (Lambda)   │ (None, 64)        │          0 │ text_vectorizati… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 2)         │          0 │ lower_input[0][0… │
│ (Concatenate)       │                   │            │ upper_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ poly_feats          │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 64, 32)    │        832 │ cast_int[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 64)        │          0 │ cast_int[0][0]    │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 16)        │         48 │ concatenate_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 32)        │        224 │ poly_feats[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 32)        │      8,320 │ embedding_4[0][0… │
│                     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 8)         │        136 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 16)        │        528 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_7       │ (None, 56)        │          0 │ lstm_3[0][0],     │
│ (Concatenate)       │                   │            │ dense_13[0][0],   │
│                     │                   │            │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 64)        │      3,648 │ concatenate_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 32)        │      2,080 │ dense_16[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pred (Dense)        │ (None, 1)         │         33 │ dense_17[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,849 (61.91 KB)

 Trainable params: 15,849 (61.91 KB)

 Non-trainable params: 0 (0.00 B)

In [60]:
for batch in train_ds.take(1):
    preds = model(batch[0])
    tf.debugging.check_numerics(preds, "NaN in forward pass")
print("OK: no NaNs.")


/Users/sbhat/Library/Python/3.9/lib/python/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['func_input', 'lower_input', 'upper_input', 'poly_feats']. Received: the structure of inputs={'func_input': '*', 'lower_input': '*', 'upper_input': '*', 'poly_feats': '*'}
  warnings.warn(
2025-06-11 18:46:42.390241: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input [ 0.8609947   0.04043978 -0.97803974  1.0738746   0.11940092  0.4024193
 -2.0231621   1.1411254  -1.579455    1.3046771   0.80718976  1.1724427
 -1.079054    0.1066768   0.74240077 -0.20810372 -1.9205327   0.45257387
 -0.6982727   0.41129914 -1.3326316   1.4841722   0.5357561   0.8715156
 -0.7543707  -0.04486049 -1.5966372  -0.10742038 -0.89840156  0.50983787
  0.40626112 -1.7293514   0.85264087 -1.3316401  -0.18863598 -0.21218397
 -0.07568236  1.0066949   0.6086284   0.85490984  1.0516126   1.0985659
  0.56583285 -0.79922205  1.2638117   1.137226   -0.1321635  -1.6166573
  0.05884328  0.7161276   1.2390001  -1.8206077   0.6511608  -1.5475467
  0.17379117 -0.8841981   1.1203841   0.46256346 -1.4833823  -1.0562903
 -1.1731285   0.10466114  1.5633659   0.5470087 ]. Expected shape (None, 6), but input has incompatible shape (64,)[0m

Arguments received by Functional.call():
  • inputs={'f

In [ ]:
# Cell 10: Training & Evaluation
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[TerminateOnNaN()]
)
val_loss, val_mae = model.evaluate(val_ds)
print(f"Val Huber={val_loss:.4f}, MAE(norm)={val_mae:.4f}")


In [67]:
# Cell 11 (revised): Safe Inference with NaN guard

import re, math
import numpy as np
import sympy as sp
import mpmath as mp

# reuse x, l_mean, l_std, u_mean, u_std, y_mean, y_std,
#       poly_feats_cached, model, INT_TOL from above

def safe_integral(func_str, a, b, tol=INT_TOL):
    # normalize inputs
    ln = (a - l_mean) / l_std
    un = (b - u_mean) / u_std
    pf = np.array(poly_feats_cached(func_str), dtype=np.float32)[None]

    # NN prediction
    func_arr = np.array([func_str], dtype=object)
    l_arr    = np.array([[ln]], dtype=np.float32)
    u_arr    = np.array([[un]], dtype=np.float32)

    r_n = model.predict([func_arr, l_arr, u_arr, pf], verbose=0)[0,0]
    p_nn = r_n * y_std + y_mean

    # if it's a valid number and rounds to an integer, return that
    if not np.isnan(p_nn) and abs(p_nn - round(p_nn)) < tol:
        return round(p_nn)

    expr_str = func_str.replace('^','**')

    # exact Sympy
    try:
        expr = sp.sympify(expr_str)
        return float(sp.integrate(expr, (x, a, b)))
    except:
        pass

    # numeric mpmath
    try:
        expr = sp.sympify(expr_str, convert_xor=True)
        f_mp = sp.lambdify(x, expr, 'mpmath')
        return float(mp.quad(f_mp, [a, b]))
    except:
        pass

    # fallback trapezoid rule
    safe_str = re.sub(r'(?<=\d)(?=[A-Za-z\(])', '*', expr_str)
    def f(v): return eval(safe_str, {"x": v, **math.__dict__})
    xs = np.linspace(a, b, 2000)
    ys = [f(v) for v in xs]
    return float(np.trapz(ys, xs))

# Interactive prompt
def predict_interactive():
    f = input("Function (e.g. (x^3)-1): ")
    a = float(input("Lower bound: "))
    b = float(input("Upper bound: "))
    print("Result ≃", safe_integral(f, a, b))

predict_interactive()


Result ≃ nan


<string>:1: RuntimeWarning: invalid value encountered in scalar divide
